* Prepare a prediction model for profit of 50_startups data.
Do transformations for getting better predictions of profit and
make a table containing R^2 value for each prepared model.

* R&D Spend -- Research and devolop spend in the past few years
* Administration -- spend on administration in the past few years
* Marketing Spend -- spend on Marketing in the past few years
* State -- states from which data is collected
* Profit  -- profit of each state in the past few years


In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
from statsmodels.graphics.regressionplots import influence_plot
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
startup = pd.read_csv("50_Startups.csv")
startup.head

# EDA

In [ ]:
startup.info()

In [ ]:
startup['State']=startup['State'].astype('category')

startup.info()

* cheack the duplicate row

In [ ]:
startup[startup.duplicated()].shape  ### here no duplicate row

In [ ]:
startup[startup.duplicated()]

### Rename the column name


In [ ]:
startup1 = startup.rename({'R&D Spend':'rd',"Marketing Spend":'ms',"Administration":'ad','Profit':'pr'}, axis = 1)

In [ ]:
startup1

# # Corelation Analysis

In [ ]:
startup1.corr()

### scatter plot between variables along with histogram 

In [ ]:
sns.pairplot(startup1)

### preparing a model

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
model1 = smf.ols('pr~rd+ad+ms',data=startup1).fit()

### Model testing

In [ ]:
model1.summary()

###  build simple linear regration and multiple linear regration model for insufisent varibles 'ad' and 'ms'

###  simple linear regration model

In [ ]:
sta = smf.ols('pr~ad',data = startup1).fit()
sta.summary()

In [ ]:
sta = smf.ols('pr~ms',data= startup1).fit()
sta.summary()

In [ ]:
sta = smf.ols('pr~ad+ms',data = startup1).fit()
sta.summary()

## Model Validation Technique
#### Two Techniques 1) Collinearity 2) Residual analyasis
##### 1) Colliniarity 
the collinearity cheche for three ways 
1) scatter plot matrix 
2) correlation matrix
3) Variance Inflation Factor (VIF)

* using VIF cheack the colliniarity

In [ ]:
rsq_rd =smf.ols('rd~ad+ms',data = startup1).fit().rsquared
vif_rd = 1/(1-rsq_rd)

In [ ]:
rsq_ad = smf.ols('ad~rd+ms',data = startup1).fit().rsquared
vif_ad = 1/(1-rsq_rd)

In [ ]:
rsq_ms = smf.ols('ms~ad+rd',data = startup1).fit().rsquared
vif_ms = 1/(1-rsq_ms)

###### Sorting VIF value in data frame

In [ ]:
df = {'variables':['rd','ad','ms'],'vif':[vif_rd,vif_ad,vif_ms]}
dataframe = pd.DataFrame(df)
dataframe

* here all VIF values less than 20 or 10 there for the no colliniarity on this model so all variables in regration equation

In [ ]:
final = smf.ols('pr~rd+ad',data = startup1).fit()
(final.rsquared, final.aic)

In [ ]:
final = smf.ols('pr~rd+ms',data = startup1).fit()
(final.rsquared,final.aic)

# Residual Analysis
### Test for Normality of Residuals (Q-Q Plot)

In [ ]:
res = model1.resid
res
res.mean()

In [ ]:
import statsmodels.api as sm
qqplot=sm.qqplot(model1.resid,line='q') 
plt.title("Normal Q-Q plot of residuals")
plt.show()

In [ ]:
list(np.where(model1.resid>10))

### Residual Plot for Homoscedasticity

In [ ]:
def get_standardize_value(vals):
    return(vals - vals.mean())/vals.std()

In [ ]:
plt.scatter(get_standardize_value(model1.fittedvalues),get_standardize_value(model1.resid))
plt.title("resiual plot")
plt.xlabel("standardized fitted values")
plt.ylabel("Standardized residules values")
plt.show()

In [ ]:
startup1.head()

### Residual Vs Regressors

In [ ]:
fig = plt.figure(figsize=(15,8))
fig =sm.graphics.plot_regress_exog(model1,'rd',fig = fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model1,'ad',fig = fig)
plt.show()

In [ ]:
fig = plt.figure(figsize = (15,8))
fig = sm.graphics.plot_regress_exog(model1,'ms',fig=fig)
plt.show()

#  model detection dignostic

#### Detecting Influencers/Outliers
* there are two tequnecs to cheaque the model detection model
* 1) Cook's Distance
* 2) Leverage value

### 1) Cook’s Distance
                        * If the Cook's distance > 1, then it's an outlier
                        * get influencer using calculate Cook's distance

In [ ]:
model_influence = model1.get_influence()
(c,_)=model_influence.cooks_distance

In [ ]:
fig = plt.subplots(figsize = (20,7))
plt.stem(np.arange(len(startup)),np.round(c,3))
plt.show()

###### index value of influencer where c > 0.25

In [ ]:
np.argmax(c),np.max(c)

#### high influence points: Points beyond Leverage_cutoff value are influencers

In [ ]:
from statsmodels.graphics.regressionplots import influence_plot
influence_plot(model1)
plt.show()

In [ ]:
k = startup.shape[1]
n = startup.shape[0]
leverage_cutoff = 3*((k+1)/n)
leverage_cutoff

In [ ]:
startup[startup1.index.isin([49])]

In [ ]:
startup1.head()

### improving the model 

In [ ]:
startup_new = startup1.drop(startup.index[[49]],axis= 0).reset_index()

In [ ]:
startup_new

In [ ]:
startup1 = startup_new.drop(['index'],axis=1)

In [ ]:
startup1.tail()

In [ ]:
startup2 = startup1.rename({'R&D Spend':'rd',"Marketing Spend":'ms',"Administration":'ad','Profit':'pr'}, axis = 1)

In [ ]:
startup2.head()

### building our final model

In [ ]:
final_ml_ad= smf.ols('pr~rd+ad',data = startup2).fit()
(final_ml_ad.rsquared,final_ml_ad.aic)

In [ ]:
final_ml_ad= smf.ols('pr~rd+ms',data = startup2).fit()
(final_ml_ad.rsquared,final_ml_ad.aic)

In [ ]:
final = smf.ols('pr~rd+ad+ms',data= startup2).fit()
(final.rsquared,final.aic)

In [ ]:
startup2

# model prediction

In [ ]:
new_startup = pd.DataFrame({'rd':50000,'ad':80000,'ms':155000},index = [1])

In [ ]:
new_startup

In [ ]:
# manual prdiction of profit
final.predict(new_startup)

In [ ]:
d2={'Prep_Models':['Model','Final_Model'],'Rsquared':[model1.rsquared,final.rsquared]}
table=pd.DataFrame(d2)
table


### Consider only the below columns and prepare a prediction model for predicting Price.

Corolla<-Corolla[c("Price","Age_08_04","KM","HP","cc","Doors","Gears","Quarterly_Tax","Weight")]

 

Model -- model of the car
Price  -- Offer Price in EUROs	
Age_08_04 -- Age in months as in August 2004	
Mfg_Month -- Manufacturing month (1-12)	
Mfg_Year	-- Manufacturing Year
KM -- Accumulated Kilometers on odometer
Fuel_Type	 -- Fuel Type (Petrol, Diesel, CNG)
HP -- Horse Power
Met_Color	 -- Metallic Color?  (Yes=1, No=0)
Color -- Color (Blue, Red, Grey, Silver, Black, etc.)
Automatic	-- Automatic ( (Yes=1, No=0)
cc -- Cylinder Volume in cubic centimeters
Doors -- Number of doors
Cylinders	-- Number of cylinders
Gears -- Number of gear positions
Quarterly_Tax -- Quarterly road tax in EUROs
Weight -- Weight in Kilograms
Mfr_Guarantee -- Within Manufacturer's Guarantee period  (Yes=1, No=0)
BOVAG_Guarantee -- BOVAG (Dutch dealer network) Guarantee  (Yes=1, No=0)
Guarantee_Period -- 	Guarantee period in months
ABS -- Anti-Lock Brake System (Yes=1, No=0)
Airbag_1 -- Driver_Airbag  (Yes=1, No=0)
Airbag_2 -- Passenger Airbag  (Yes=1, No=0)
Airco -- Airconditioning  (Yes=1, No=0)
Automatic_airco -- Automatic Airconditioning  (Yes=1, No=0)
Boardcomputer -- Boardcomputer  (Yes=1, No=0)
CD_Player -- CD Player  (Yes=1, No=0)
Central_Lock -- Central Lock  (Yes=1, No=0)
Powered_Windows -- Powered Windows  (Yes=1, No=0)
Power_Steering -- Power Steering  (Yes=1, No=0)
Radio -- Radio  (Yes=1, No=0)
Mistlamps	-- Mistlamps  (Yes=1, No=0)
Sport_Model -- Sport Model  (Yes=1, No=0)
Backseat_Divider -- Backseat Divider  (Yes=1, No=0)
Metallic_Rim --Metallic Rim  (Yes=1, No=0)
Radio_cassette -- Radio Cassette  (Yes=1, No=0)
Tow_Bar -- Tow Bar  (Yes=1, No=0)




In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.regressionplots import influence_plot
import statsmodels.formula.api as smf

In [ ]:
data = pd.read_csv('ToyotaCorolla.csv')

In [ ]:
data.head(1)

# EDA

###  Corolla<-Corolla[c("Price","Age_08_04","KM","HP","cc","Doors","Gears","Quarterly_Tax","Weight")]

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.head(1)

In [ ]:
data1 = pd.concat([data.iloc[:,2:4],data.iloc[:,6:7],data.iloc[:,8:9],data.iloc[:,12:14],data.iloc[:,15:18]],axis=1)

In [ ]:
data1

In [ ]:
data1.info()

###  Duplicates

In [ ]:
data1[data.duplicated()]  ### there is no duplicste values

### Rename

In [ ]:
data2 = data1.rename({'Age_08_04':'Age','Quarterly_Tax':'QT','cc':'CC'},axis = 1)

In [ ]:
data2

In [ ]:
data2[data2.duplicated()]

In [ ]:
data3 = data2.drop_duplicates().reset_index(drop = True)
data3

In [ ]:
data3.describe()

#  Corilation Matrix


In [ ]:
data3.corr()

In [ ]:
# scatter plot between variables along with histogram
sns.set_style(style='darkgrid')
sns.pairplot(data3)

### Preparing a model 

In [ ]:
import statsmodels.formula.api as smf
model = smf.ols('Price~Age+KM+HP+CC+Doors+Gears+QT+Weight',data = data3).fit()
model.summary()

### Build a simple linear regration and Multiple linear regration model for insuficent variable 'CC' and 'Doors'

In [ ]:
model_CC = smf.ols('Price~CC',data=data3).fit()
model_CC.summary()

In [ ]:
model_Doors = smf.ols('Price~ Doors',data=data3).fit()
model_Doors.summary()

In [ ]:
model_CC_Doors = smf.ols('Price ~ CC + Doors',data = data3).fit()
model_CC_Doors.summary()

# Model Validation Technique
### Cheack Colinearity using VIF

In [ ]:
rsq_age = smf.ols('Age ~ KM + HP + CC + Doors + Gears + QT + Weight',data=data3).fit().rsquared
VIF_age = 1/(1-rsq_age)

rsq_KM = smf.ols('KM ~ Age + HP + CC + Doors + Gears + QT + Weight',data = data3).fit().rsquared
VIF_KM = 1/(1-rsq_KM)

rsq_HP = smf.ols('HP ~ CC + Doors + Gears + QT + Weight + Age + KM',data = data3).fit().rsquared
VIF_HP = 1/(1-rsq_HP)

rsq_CC = smf.ols('CC ~ Doors + Gears + QT + Weight + Age + KM + HP',data = data3).fit().rsquared
VIF_CC = 1/(1-rsq_CC)

rsq_Doors = smf.ols('Doors ~ CC + Gears + QT + Weight + Age + KM + HP',data = data3).fit().rsquared
VIF_Doors = 1/(1-rsq_Doors)

rsq_Gears = smf.ols('Gears ~ Doors + CC  + QT + Weight + Age + KM + HP',data = data3).fit().rsquared
VIF_Gears = 1/(1-rsq_Gears)

rsq_QT = smf.ols('QT ~ Doors + CC + Gears + Weight + Age + KM + HP',data = data3).fit().rsquared
VIF_QT = 1/(1-rsq_QT)

rsq_Weight = smf.ols('Weight ~ Age + KM + HP + CC + Doors + Gears + QT ',data = data3).fit().rsquared
VIF_Weight = 1/(1-rsq_Weight)



In [ ]:
d1 = {'varibles':['age','km','hp','cc','doors','gears','qt','weight'],'VIF':[VIF_age,VIF_KM,VIF_HP,VIF_CC,VIF_Doors,VIF_Gears,VIF_QT,VIF_Weight]}
d2 = pd.DataFrame(d1)
d2

### All VIF values are less than 20 or 10 there for no collinearity for this model so its regration equation

###  Build final model

In [ ]:
final_ml_CC = smf.ols('Price~Age+KM+HP+CC+Gears+QT+Weight',data = data3).fit()
(final_ml_CC.rsquared,final_ml_CC.aic)

In [ ]:
final_ml_Doors = smf.ols('Price~Age+KM+HP+Doors+Gears+QT+Weight',data = data3).fit()
(final_ml_Doors.rsquared,final_ml_Doors.aic)

## Model validation tachniques

### Residual analysis 

In [ ]:
res = model.resid
res
res.mean()

In [ ]:
import statsmodels.api as sm

In [ ]:
qqplot = sm.qqplot(res,line='q')
plt.show()

* In the reside mean is not equal to 0 and this plot there are some points are not perticular line beacuse of this points we not conclude that your error are  normaly distributed

In [ ]:
list(np.where(model.resid>10))

### Residual plot for Homoscedasticity 

In [ ]:
def get_standardize_value(vals):
    return(vals - vals.mean())/vals.std()

In [ ]:
plt.scatter(get_standardize_value(model.fittedvalues),get_standardize_value(model.resid))
plt.title("Residul plot")
plt.xlabel("standared fitted values")
plt.ylabel("standered residul values")

### Residual vs Regrassor

In [ ]:
fig = plt.figure(figsize = (15,8))
fig = sm.graphics.plot_regress_exog(model,'Age',fig = fig)
plt.show()

In [ ]:
fig = plt.figure(figsize = (15,8))
fig = sm.graphics.plot_regress_exog(model,'KM',fig = fig)
plt.show()

In [ ]:
fig = plt.figure(figsize = (15,8))
fig = sm.graphics.plot_regress_exog(model,'HP',fig = fig)
plt.show()

In [ ]:
fig = plt.figure(figsize = (15,8))
fig = sm.graphics.plot_regress_exog(model,'CC',fig = fig)
plt.show()

In [ ]:
fig = plt.figure(figsize = (15,8))
fig = sm.graphics.plot_regress_exog(model,'Doors',fig = fig)
plt.show()

In [ ]:
fig = plt.figure(figsize = (15,8))
fig = sm.graphics.plot_regress_exog(model,'Gears',fig = fig)
plt.show()

In [ ]:
fig = plt.figure(figsize = (15,8))
fig = sm.graphics.plot_regress_exog(model,'QT',fig = fig)
plt.show()

In [ ]:
fig = plt.figure(figsize = (15,8))
fig = sm.graphics.plot_regress_exog(model,'Weight',fig = fig)
plt.show()

## Model Deletion Diagnostics

In [ ]:
model_influence = model.get_influence()
(c,_)= model_influence.cooks_distance

In [ ]:
c

In [ ]:
fig = plt.subplots(figsize=(20,7))
plt.stem(np.arange(len(data3)),np.round(c,3))
plt.xlabel('Row Index')
plt.ylabel('cooks distance')
plt.show

In [ ]:
 #Index and value of influencer where C>0.5
np.argmax(c) , np.max(c)

### High influance point

In [ ]:
from statsmodels.graphics.regressionplots import influence_plot

In [ ]:
influence_plot(model)
plt.show()

In [ ]:
k = data3.shape[1]
n = data3.shape[0]
leverage_cutoff = 3*((k+1)/n)
leverage_cutoff

In [ ]:
data3[data3.index.isin([80])]

In [ ]:
data3

### Improving the model

In [ ]:
data4 = data3.drop(data3.index[[80]],axis = 0).reset_index()
data4

In [ ]:
data5 = data4.drop(['index'],axis=1)
data5

### Build our final model

In [ ]:
final_model_CC = smf.ols('Price~Age+KM+HP+CC+Gears+QT+Weight',data = data5).fit()
(final_model_CC.rsquared,final_model_CC.aic)

In [ ]:
final_model_Doors = smf.ols('Price~Age+KM+HP+Doors+Gears+QT+Weight',data = data5).fit()
(final_model_Doors.rsquared,final_model_Doors.aic)

In [ ]:
final_model = smf.ols('Price~Age+KM+HP+CC+Doors+Gears+QT+Weight',data = data5).fit()
final_model.summary()

In [ ]:
final_model.rsquared

###  Predict for new data

In [ ]:
new_data = pd.DataFrame({'Age':25,'KM':50000,'HP':55,'CC':2500,'Doors':3,'Gears':6,'QT':200,'Weight':1660},index=[1])

In [ ]:
new_data

In [ ]:
final_model.predict(new_data)

In [ ]:
d2={'Prep_Models':['Model','Final_Model'],'Rsquared':[model.rsquared,final_model.rsquared]}
table=pd.DataFrame(d2)
table
